# MODIS16A2数据重采样

尝试对MODIS16A2数据进行重采样。根据附带的数据评估对响应数据进行赋权后取平均。

## 平均算法

MODIS数据质量有10个等级。记等级权重为服从某种分布的序列

$$p(l), l=1,2,3,...,10$$

则

$$\sum p(l)=1$$

先从单个格点开始。记蒸散量数据的时间序列数据$\{x_n\}$，$n$是其在时间序列上的位置，其等级为$l(x_n)$，故权重为$p(l(x_n))$，这里简记为$p_n$。则均值计算方法为

$$ \overline{x}=\frac{\sum_{j=1}^{n}x_j\cdot p_j}{\sum_{j=1}^{n} p_j} $$

直接将$\{x_n\}$看作数组$x$，可以利用`numpy`实现并行计算

$$ 
\begin{align}
\overline{x}&=\frac{x\cdot p}{\mathrm{sum}(p)} \\
p&=l(x)

\end{align}
$$

以上可以得到单个格点的经过加权重采样的时间序列数据。接下来，将对其转化成三维数组：其中第一维是时间，其余两个维度定位格点数据。


$$
\begin{align}

\overline{X}&= \frac{X\cdot P}{\mathrm{sum}(P,\mathrm{axis=}0)} \\
P&=L(X)

\end{align}
$$
其中，$X$为三维数组，可以看作一系列蒸散量矩阵排列而成的数组；$P$与其类似，是权重矩阵数组。`axis=0`使得`numpy`只在时间维度上进行求和。

## 确定数据等级权重序列

对于$p(l)$，整数$l$在$0\le l \le 10$时，数据可用性依次下降。这里认为规定下一等级的数据可信度时上一等级的一半：

$$ p(l+1)=\frac{1}{2}p(l) $$

故$p(l)$构成等比数列，且

$$ \sum_{l=1}{10}p(l)=1 $$

易得
$$p(l)=\frac{2^{10-l}}{2^{10}-1}$$

下面输出这个序列

In [1]:
print([(2**(10-i))/(2**10-1) for i in range(1,11)])

[0.5004887585532747, 0.25024437927663734, 0.12512218963831867, 0.06256109481915934, 0.03128054740957967, 0.015640273704789834, 0.007820136852394917, 0.0039100684261974585, 0.0019550342130987292, 0.0009775171065493646]
